In [ ]:
%cd ~
import pyfits,glob,time,scipy
import scipy.interpolate
from scipy.special import erf
from pylab import *
#from numpy import *
#from matplotlib.pyplot import *
from subprocess import call
from IPython import parallel
from scipy.optimize import fmin_powell

topdir='/Users/cslage/Research/LSST/code/GUI/'
thedir=topdir+'profiles/'
%cd $thedir

configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'

%matplotlib inline

In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()
    
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

In [ ]:
# This definition runs sextractor in parallel and can be given to IPython's parallel map function along with
#   the file list
def ovsub_runsex_makereg(fitsfilename):
    import pyfits
    import numpy as np
    from subprocess import call
    topdir='/Users/cslage/Research/LSST/code/GUI/'
    thedir=topdir+'profiles'
    %cd $thedir

    configfile=topdir+'sextractor/default-array_dither.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    maskdir=topdir+'sextractor/masks/'

    def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
        overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
        image=image[:y_overscan_start,:x_overscan_start]
        finalimg=image-np.matrix(np.median(overscan,axis=1)).T*np.ones(np.shape(image)[1])
        return finalimg

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
    for i in range(1,17):
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname)
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022)   # cut off the overscan
        outname=fitsfilename[:-5]+extname+'.fits'
        pyfits.writeto(outname,overscansubimg,clobber=True)
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat'])
        make_reg_from_ldac(outname+'.cat','NUMBER')

In [ ]:
zfilelist=sort(glob.glob(thedir+'114-04_spot-30um_light_3??_20150709??????.fits'))
zfilelist

In [ ]:
for fitsfilename in zfilelist[49:-1]: 
    tfile1=time.time() 
    for i in [7]:
        tstart=time.time() 
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME'] 
        img=pyfits.getdata(fitsfilename,extname) 
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
        # cut off the overscan 
        outname=fitsfilename[:-5]+extname+'.fits' 
        pyfits.writeto(outname,overscansubimg,clobber=True) 
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat']) 
        make_reg_from_ldac(outname+'.cat','NUMBER') 
        tend=time.time() 
        print extname+" time: "+str(tend-tstart)[:4] 
        print "Time taken for file "+str(fitsfilename[-23:-20])+": "+str(time.time()-tfile1)

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny):
    spotlist = []
    hdr=pyfits.getheader(fitsfilename,segmentnumber)
    extname = hdr['EXTNAME']
    img=pyfits.getdata(fitsfilename,extname) 
    overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
    catname=fitsfilename[:-5]+extname+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    for j in range(numspots):
        line = catlines[j]
        xcoord = float(line.split()[1])
        ycoord = float(line.split()[2])
        xint = int(xcoord-0.5)
        yint = int(ycoord-0.5)
        xoffset = xcoord - xint - 1.0
        yoffset = ycoord - yint - 1.0     
        xmin = xint - int(nx/2)
        xmax = xint + int(nx/2) + 1
        ymin = yint - int(ny/2)
        ymax = yint + int(ny/2) + 1
        small_img = overscansubimg[ymin:ymax, xmin:xmax]
        spotlist.append([(xoffset,yoffset),small_img])
        
    return spotlist

def FOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([nx,ny])
   
    for spot in range(len(spotlist)):
        for ii in range(nx):
            for jj in range(ny):
                xl = ii - int(nx/2) - spotlist[spot][0][0] - 0.5
                xh = xl + 1.0
                yl = jj - int(nx/2) - spotlist[spot][0][1] - 0.5
                yh = yl + 1.0
                #print "ii = %d, jj = %d, imga = %.1f"%(ii,jj,small_img[jj,ii])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[jj,ii] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                try:
                    fom += square(area[jj,ii]-spotlist[spot][1][jj,ii])
                except:
                    continue
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom


In [ ]:
nx = 9
ny = 9
numspots = 100
results = []
segmentnumber = 7

for fitsfilename in zfilelist:
    param0 = [50000.0, 1.06, 1.11]
    
    spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny)

    Result = fmin_powell(FOM, param0)
    print fitsfilename
    print Result
    
    results.append(Result)


In [ ]:

results[94]


In [ ]:
figure()
imaxs = []
sigmaxs = []
sigmays = []
for i,result in enumerate(results[0:93]):
    imaxs.append(result[0])
    sigmaxs.append(result[1])
    sigmays.append(result[2])
    if result[2] < 1.06:
        print "I = %d, Imax = %.1f, sigmax = %.3f, sigmay = %.3f"%(i,result[0],result[1],result[2])
scatter(imaxs, sigmaxs, color = 'blue', lw = 2, label = 'Sigma-x')
scatter(imaxs, sigmays, color = 'green', lw = 2, label = 'Sigma-y')
xlabel('Central Peak(ADUs)')
ylabel('Sigma (Pixels)')
legend(loc= 'lower right')
savefig("Forward_Model_Spots_Powell.png")

In [ ]:
figure()
testn = 92
spot = 87
spotlist = BuildSpotList(zfilelist[testn], segmentnumber, numspots, nx, ny)
Result = results[testn]
Imax = Result[0]
sigmax = Result[1]
sigmay = Result[2]
seqno = int(zfilelist[testn].split("_")[3])
suptitle=("Sequence # %d, Spot # %d"%(seqno,spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist[spot][0][0] - 0.5
        xh = xl + 1.0
        yl = jj - int(nx/2) - spotlist[spot][0][1] - 0.5
        yh = yl + 1.0
        area[jj,ii] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)

subplots_adjust(wspace = 1.0)
subplot(1,3,1)
imshow(spotlist[spot][1],interpolation="None")
#colorbar()
subplot(1,3,2)
plot(spotlist[spot][1][:,int(nx/2)], lw = 2, label="Data")
plot(area[:,int(nx/2)], lw = 2, label="Model")
xlabel("X (Pixels)")
subplot(1,3,3)
plot(spotlist[spot][1][int(ny/2),:], lw = 2, label="Data")
plot(area[int(ny/2),:], lw = 2, label="Model")
xlabel("Y (Pixels)")
legend(loc = (-4.0,0.8))
savefig("Typical_Fit_Powell")


In [ ]:
print len(zfilelist)
print len(results)

In [ ]:
from scipy import stats
figure()
exp_current=[]
intensities=[]
for i,fitsfilename in enumerate(zfilelist[0:93]):
    try:
        exptime=pyfits.getheader(fitsfilename,0)['EXPTIME'] 
        mondiode=pyfits.getheader(fitsfilename,0)['MONDIODE'] 
        exp_current.append(float(exptime) * mondiode / (1.0E-9))
        intensity=results[i][0]
        intensities.append(intensity/1000.0)
    except:
        continue
slope, intercept, r_value, p_value, std_err = stats.linregress(exp_current,intensities)

xplot=linspace(0.0,7.0,100)
yplot = slope * xplot + intercept
scatter(exp_current,intensities)
plot(xplot, yplot, color='red')
text(0.5,300,"Linear Fit R^2 = %.5f"%r_value)
xlabel("Exp Time * Monitor Diode (nA-sec)")
ylabel("Modeled Peak Intensity (kADU)")
xlim(0.0, 7.0)
ylim(0.0,350.0)
savefig("Intensity_Check_Powell.png")